In [1]:
# Q1.
import pyspark
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.master("local[*]").appName("homework").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 17:39:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
pyspark.__version__


'3.3.2'

In [7]:
#Q2

URL = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz"


In [ ]:
# I've ran these below 2 commands in my terminal to get data in my desired location
#1.  mkdir -p data/raw/fhvhv/2021/06
#2. wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz -O data/raw/fhvhv/2021/06/fhvhv_tripdata_2021_06.csv.gz

# TODO:
# we need to write a pyspark code to read csv data from URL or atleast save data from URL to desired local location

In [5]:
df_fhvhv = spark_session.read.option("header", "true").csv("../notebooks/data/raw/fhvhv/2021/06/")

In [6]:
df_fhvhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
# we can see that the schema has all strings, so we need to read using pandas to get correct spark alike schema and then 
# use that schema to read data into spark DF again

import pandas as pd

pd_df = pd.read_csv('../notebooks/data/raw/fhvhv/2021/06/fhvhv_tripdata_2021_06.csv.gz', nrows=100) #as we just need schema, we can even read 1 or 10 rows as well



In [12]:
pd_df.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [22]:
pd_df.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [13]:
spark_session.createDataFrame(pd_df).schema

TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [14]:
import numpy as np

In [23]:
# seems like we have some NaN values in the Affiliated_base_number column
len(pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)])

34

In [25]:
# so we replace them with the string "NaN"
pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)] = "NaN"

/tmp/ipykernel_3723/2845800174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)] = "NaN"


In [26]:
pd_df.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [27]:
spark_session.createDataFrame(pd_df).schema

/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [37]:
#copy the above schema into some editor eg:vscode and add some spark.sql.types like below
from pyspark.sql import types

fhvhv_schema = types.StructType([types.StructField('dispatching_base_num', types.StringType(), True),
 types.StructField('pickup_datetime', types.TimestampType(), True),
 types.StructField('dropoff_datetime', types.TimestampType(), True),
 types.StructField('PULocationID', types.IntegerType(), True),
 types.StructField('DOLocationID', types.IntegerType(), True),
 types.StructField('SR_Flag', types.StringType(), True),
 types.StructField('Affiliated_base_number', types.StringType(), True)])

In [38]:
input_path = f'../notebooks/data/raw/fhvhv/2021/06/'
output_path = f'../notebooks/data/pq/fhvhv/2021/06/'

df_fhvhv = spark_session.read.option('header', 'true').schema(fhvhv_schema).csv(input_path)

In [39]:
# as asked in the Q2
#Repartition it to 12 partitions and save it to parquet.
df_fhvhv.repartition(12).write.parquet(output_path, mode='overwrite')

In [40]:
ls ../notebooks/data/pq/fhvhv/2021/06/ -l

total 290748
-rw-r--r-- 1 jagadish jagadish        0 Mar 15 18:17 _SUCCESS
-rw-r--r-- 1 jagadish jagadish 24801840 Mar 15 18:17 part-00000-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24799764 Mar 15 18:17 part-00001-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24813665 Mar 15 18:17 part-00002-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24816657 Mar 15 18:17 part-00003-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24807133 Mar 15 18:17 part-00004-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24800815 Mar 15 18:17 part-00005-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24812906 Mar 15 18:17 part-00006-18cfd5d7-176d-4253-9eae-246f5433978a-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24809398 Mar 15 18:17 part-00007-18cfd5d7-176d-

In [ ]:
# avg MB size of these partitions are 25MB

In [41]:
# Q3
df_fhvhv.head(5)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 16, 16), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 21, 14), PULocationID=32, DOLocationID=254, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 27, 1), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 42, 11), PULocationID=240, DOLocationID=127, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 46, 8), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 53, 45), PULocationID=127, DOLocationID=235, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02510', pickup_datetime=datetime.d

In [42]:
df_fhvhv.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [43]:
df_pq_fhvhv = spark_session.read.parquet('../notebooks/data/pq/fhvhv/2021/06/')

In [44]:
df_pq_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02889|2021-06-04 20:51:44|2021-06-04 21:10:12|         239|         158|      N|                B02889|
|              B02800|2021-06-04 15:50:15|2021-06-04 16:19:29|          75|         116|      N|                  null|
|              B02510|2021-06-02 21:03:38|2021-06-02 21:10:12|         167|         168|      N|                  null|
|              B02867|2021-06-02 12:51:57|2021-06-02 13:05:09|         151|         142|      N|                B02867|
|              B02869|2021-06-21 09:51:45|2021-06-21 10:09:17|         106|          65|      N|                B02869|
|              B02764|2021-06-02 13:27:0

In [54]:
df_results = df_pq_fhvhv.select('dispatching_base_num') \
    .filter((df_pq_fhvhv.pickup_datetime >= '2021-06-15 00:00:00') & (df_pq_fhvhv.pickup_datetime < '2021-06-16 00:00:00'))

In [55]:
df_results.count()

452470